In [37]:
from google.colab import drive
drive.mount('/content/drive')

In [38]:
!pip install pandas

In [39]:
import pandas as pd
import numpy as np

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.svm import LinearSVC

In [41]:
data=pd.read_csv('/content/drive/MyDrive/NER_NLP_PROJ/Tulu_NER - Tulu_NER.csv')
#data_dev=pd.read_excel('')
data

,Words,NER Tag
0,ದಕ್ಷಿಣ,B-LOC
1,ಕನ್ನಡ,I-LOC
2,ಜಿಲ್ಲೆಗ್,B-GPE
3,ತುರ್ತು,B-MED
4,ಆಕ್ಸಿಜನ್,B-MED
...,...,...
20202,ದಕ್ಷಿಣ,B-LOC
20203,ಕನ್ನಡ,I-LOC
20204,ಜಿಲ್ಲಾಡಳಿತಗ್,B-GPE
20205,ಕೊರ್ತೆರ್,O


In [42]:
data.head()

,Words,NER Tag
0,ದಕ್ಷಿಣ,B-LOC
1,ಕನ್ನಡ,I-LOC
2,ಜಿಲ್ಲೆಗ್,B-GPE
3,ತುರ್ತು,B-MED
4,ಆಕ್ಸಿಜನ್,B-MED


In [43]:
#data_dev.head()

In [44]:
data.columns.unique()

Index(['Words', 'NER Tag'], dtype='object')

In [46]:
data['NER Tag'].value_counts()

O         9642
I-ORG      871
B-ACD      861
B-LOC      755
B-ORG      738
B-ROL      716
I-PER      656
B-DATE     644
B-PER      637
B-POL      558
B-NUM      557
B-MED      556
B-FAM      304
I-ACD      280
I-MED      275
B-GPE      269
B-LEF      263
B-FUN      231
I-ROL      227
I-LOC      222
B-COM      144
I-POL      138
B-TRS      110
I-LEF       88
I-FUN       85
I-DATE      83
B-ENT       78
I-TRS       44
B-GRC       35
I-FAM       31
I-ENT       28
I-GPE       11
I-COM       11
I-GRC        9
B-SPT        7
I-NUM        7
I-SPT        4
o            4
Name: NER Tag, dtype: int64

In [47]:
#data_dev['LABEL'].value_counts()

In [48]:
len(data)

20207

In [49]:
!git clone https://github.com/libindic/syllabalizer.git
%cd syllabalizer
%run setup.py sdist

Cloning into 'syllabalizer'...
remote: Enumerating objects: 186, done.
remote: Total 186 (delta 0), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (186/186), 41.35 KiB | 901.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/syllabalizer/syllabalizer/syllabalizer


/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separ

In [50]:
!pip install dist/libindic-syllabifier*.tar.gz
!pip install libindic-utils

Processing ./dist/libindic-syllabifier-0.0.1.dev28.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-syllabifier: filename=libindic_syllabifier-0.0.1.dev28-py2.py3-none-any.whl size=8122 sha256=91744a6fbe99e6bafdf64653cd6d4cc4c0b26c79f50180db438b3a5875918ac5
  Stored in directory: /root/.cache/pip/wheels/a0/fd/80/24c45edb1a60785333ece5126c49573ab43da56a7c13f07d6a
Successfully built libindic-syllabifier
  Attempting uninstall: libindic-syllabifier
    Found existing installation: libindic-syllabifier 0.0.1.dev28
    Uninstalling libindic-syllabifier-0.0.1.dev28:
      Successfully uninstalled libindic-syllabifier-0.0.1.dev28


In [51]:
from libindic.syllabifier import Syllabifier
instance = Syllabifier()

In [52]:
def syllable(word):
    try:
        sylbls = instance.syllabify(word)
    except:
        sylbls = list(word)
    return sylbls

In [53]:
# generate word n-grams

def generate_N_grams(text, ngram=1):
    words = [word for word in text.split(" ")]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[words[i:] for i in range(0, ngram)])
    ngrams = [' '.join(ngram) for ngram in temp]
    return ngrams

In [54]:
import regex

def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
    for w in words:
        yield w

In [55]:
# generate syllable n-grams by specifying the value of n, by default ngram=1

def generate_cN_grams(word, ngram=1):
    sylbs = [sylbs for sylbs in syllable(word)]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[sylbs[i:] for i in range(0, ngram)])
    ngrams = [''.join(ngram) for ngram in temp]
    return ngrams

In [56]:
# generate syllable n-grams in the range 'from' to 'to'. by default 'from'= 1

def ngrams(text, r_to, r_from=1):
    r_from = 1
    r_to = 3
    allngrams=[]
    for word in text.split():
        temp = []
        for i in range(r_from + r_to - 1):
            temp.extend(generate_cN_grams(word, i+1))
            allngrams.append(temp)
    n_grams = [item for sublist in allngrams for item in sublist]
  # merged = list(itertools.chain(allngrams))
    return n_grams

In [57]:
def custom_analyzer(text):
    syngrams = ngrams(text,3)
    return syngrams

In [58]:
from sklearn.model_selection import train_test_split

In [68]:
data=data.dropna()

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data['Words'], data['NER Tag'], test_size=0.30, random_state=None)

In [70]:
from sklearn.model_selection import train_test_split
X_train=data['Words']
y_train=data['NER Tag']
#X_test=df_dev['text']
#y_test=df_dev['tag']

In [71]:
tfidf_vectorizer = TfidfVectorizer(analyzer=custom_analyzer)
tfidf_vectorizer.fit_transform(X_train)
train_tfidf1 =  tfidf_vectorizer.transform(X_train)
dev_tfidf1=tfidf_vectorizer.transform(X_test)

In [72]:
data = data.dropna() # drop rows with nan values
print(data)


              Words NER Tag
0            ದಕ್ಷಿಣ   B-LOC
1             ಕನ್ನಡ   I-LOC
2          ಜಿಲ್ಲೆಗ್   B-GPE
3            ತುರ್ತು   B-MED
4          ಆಕ್ಸಿಜನ್   B-MED
...             ...     ...
20202        ದಕ್ಷಿಣ   B-LOC
20203         ಕನ್ನಡ   I-LOC
20204  ಜಿಲ್ಲಾಡಳಿತಗ್   B-GPE
20205      ಕೊರ್ತೆರ್       O
20206             .       O

[20173 rows x 2 columns]


In [73]:
from sklearn import svm
clf1= svm.SVC()
clf1.fit(train_tfidf1 ,y_train)
val_pred1=clf1.predict(dev_tfidf1)

In [74]:
from sklearn.metrics import classification_report
print(classification_report(y_test, val_pred1))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-ACD       0.80      0.83      0.82       234
       B-COM       0.94      0.81      0.87        54
      B-DATE       0.91      0.92      0.91       199
       B-ENT       0.87      0.62      0.72        21
       B-FAM       0.84      0.82      0.83        92
       B-FUN       0.77      0.64      0.70        72
       B-GPE       0.80      0.90      0.85        82
       B-GRC       0.69      0.90      0.78        10
       B-LEF       0.92      0.81      0.86        80
       B-LOC       0.83      0.93      0.88       236
       B-MED       0.84      0.82      0.83       172
       B-NUM       0.97      1.00      0.98       184
       B-ORG       0.78      0.67      0.72       214
       B-PER       0.88      0.91      0.90       185
       B-POL       0.79      0.80      0.79       173
       B-ROL       0.88      0.85      0.86       213
       B-SPT       1.00      1.00      1.00         1
       B-TRS       0.87    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
from sklearn.linear_model import LogisticRegression
clf2=LogisticRegression(solver='liblinear', multi_class='ovr')
clf2.fit(train_tfidf1, y_train)
val_pred2=clf2.predict(dev_tfidf1)

In [76]:
from sklearn.metrics import classification_report
print(classification_report(y_test, val_pred2))

              precision    recall  f1-score   support

       B-ACD       0.72      0.55      0.63       234
       B-COM       0.91      0.59      0.72        54
      B-DATE       0.91      0.82      0.86       199
       B-ENT       1.00      0.05      0.09        21
       B-FAM       0.80      0.58      0.67        92
       B-FUN       0.74      0.32      0.45        72
       B-GPE       0.72      0.77      0.74        82
       B-GRC       0.00      0.00      0.00        10
       B-LEF       0.89      0.41      0.56        80
       B-LOC       0.78      0.61      0.69       236
       B-MED       0.80      0.71      0.75       172
       B-NUM       0.96      0.98      0.97       184
       B-ORG       0.67      0.39      0.49       214
       B-PER       0.81      0.58      0.68       185
       B-POL       0.76      0.54      0.64       173
       B-ROL       0.78      0.57      0.66       213
       B-SPT       0.00      0.00      0.00         1
       B-TRS       0.80    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
from sklearn.naive_bayes import MultinomialNB
clf3 = MultinomialNB(force_alpha=True)
clf3.fit(train_tfidf1, y_train)
val_pred3=clf3.predict(dev_tfidf1)

In [78]:
from sklearn.metrics import classification_report
print(classification_report(y_test, val_pred3))

              precision    recall  f1-score   support

       B-ACD       0.70      0.32      0.44       234
       B-COM       0.00      0.00      0.00        54
      B-DATE       0.92      0.58      0.71       199
       B-ENT       0.00      0.00      0.00        21
       B-FAM       1.00      0.10      0.18        92
       B-FUN       0.82      0.12      0.22        72
       B-GPE       0.66      0.43      0.52        82
       B-GRC       0.00      0.00      0.00        10
       B-LEF       0.00      0.00      0.00        80
       B-LOC       0.80      0.40      0.54       236
       B-MED       0.79      0.64      0.71       172
       B-NUM       0.96      0.96      0.96       184
       B-ORG       0.76      0.24      0.36       214
       B-PER       0.85      0.29      0.43       185
       B-POL       0.87      0.31      0.45       173
       B-ROL       0.71      0.45      0.55       213
       B-SPT       0.00      0.00      0.00         1
       B-TRS       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
from sklearn.tree import DecisionTreeClassifier
clf4 = DecisionTreeClassifier(random_state=0)
clf4.fit(train_tfidf1, y_train)
val_pred4=clf4.predict(dev_tfidf1)

In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_test, val_pred4))

              precision    recall  f1-score   support

       B-ACD       0.81      0.89      0.85       234
       B-COM       0.94      0.94      0.94        54
      B-DATE       0.93      0.98      0.95       199
       B-ENT       0.76      0.90      0.83        21
       B-FAM       0.90      0.97      0.93        92
       B-FUN       0.78      0.86      0.82        72
       B-GPE       0.77      0.94      0.85        82
       B-GRC       0.71      1.00      0.83        10
       B-LEF       0.93      0.97      0.95        80
       B-LOC       0.88      0.97      0.92       236
       B-MED       0.83      0.90      0.86       172
       B-NUM       0.97      1.00      0.99       184
       B-ORG       0.76      0.74      0.75       214
       B-PER       0.88      0.93      0.90       185
       B-POL       0.82      0.91      0.86       173
       B-ROL       0.89      0.87      0.88       213
       B-SPT       1.00      1.00      1.00         1
       B-TRS       0.94    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [81]:
from sklearn.neighbors import KNeighborsClassifier
clf5 = KNeighborsClassifier(n_neighbors=3)
clf5.fit(train_tfidf1, y_train)
val_pred5=clf5.predict(dev_tfidf1)

In [82]:
from sklearn.metrics import classification_report
print(classification_report(y_test, val_pred5))

              precision    recall  f1-score   support

       B-ACD       0.62      0.82      0.71       234
       B-COM       0.75      0.87      0.80        54
      B-DATE       0.78      0.92      0.85       199
       B-ENT       0.65      0.71      0.68        21
       B-FAM       0.76      0.83      0.79        92
       B-FUN       0.56      0.68      0.62        72
       B-GPE       0.68      0.80      0.74        82
       B-GRC       0.62      1.00      0.77        10
       B-LEF       0.78      0.85      0.81        80
       B-LOC       0.73      0.80      0.76       236
       B-MED       0.73      0.84      0.78       172
       B-NUM       0.96      0.99      0.98       184
       B-ORG       0.60      0.52      0.56       214
       B-PER       0.72      0.72      0.72       185
       B-POL       0.72      0.75      0.73       173
       B-ROL       0.79      0.74      0.77       213
       B-SPT       0.50      1.00      0.67         1
       B-TRS       0.72    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
